In [3]:
import pandas as pd
import re
from Bio import SeqIO
from IPython.display import display
from pathlib import Path
import os

# ================================
# 1️⃣ TỰ ĐỘNG TÌM PROJECT ROOT
# ================================
def find_project_root(target_folder_name="CAFA-6-Protein-Function-Function-Prediction-Kaggle"):
    path = Path.cwd()
    while path.name != target_folder_name:
        if path.parent == path:
            raise RuntimeError("❌ Không tìm thấy thư mục project gốc.")
        path = path.parent
    return path

# Nếu tên thư mục project của bạn KHÁC → sửa tại đây:
PROJ_DIR = find_project_root("CAFA-6-Protein-Function-Prediction-Kaggle")

# ================================
# 2️⃣ CẤU HÌNH ĐƯỜNG DẪN
# ================================
BASE_DIR = PROJ_DIR / "data" / "Train"
TRAIN_FASTA_PATH = BASE_DIR / "train_sequences.fasta"
TRAIN_TERMS_PATH = BASE_DIR / "train_terms.tsv"

print("📁 Project Root:", PROJ_DIR)
print("📁 Train Folder:", BASE_DIR)
print("📄 FASTA Exists:", TRAIN_FASTA_PATH.exists())
print("📄 TERMS Exists:", TRAIN_TERMS_PATH.exists())

# Nếu file không tồn tại → dừng lại
if not TRAIN_FASTA_PATH.exists():
    raise FileNotFoundError(f"❌ Không tìm thấy file FASTA: {TRAIN_FASTA_PATH}")

if not TRAIN_TERMS_PATH.exists():
    raise FileNotFoundError(f"❌ Không tìm thấy file TERMS: {TRAIN_TERMS_PATH}")


# ================================
# 3️⃣ ĐỌC FASTA + TRÍCH XUẤT FEATURES
# ================================
print("\n1. Đang đọc FASTA & trích xuất Features...")

ids = []
sequences = []
pe_list = []
taxon_list = []
reviewed_list = []

for record in SeqIO.parse(str(TRAIN_FASTA_PATH), "fasta"):
    
    # Lấy Entry ID (chuẩn UniProt)
    parts = record.id.split("|")
    clean_id = parts[1] if len(parts) >= 2 else record.id

    header = record.description
    
    # PE (Protein Existence)
    pe_match = re.search(r"PE=(\d+)", header)
    pe_val = int(pe_match.group(1)) if pe_match else 0

    # OX (Taxonomy)
    ox_match = re.search(r"OX=(\d+)", header)
    ox_val = int(ox_match.group(1)) if ox_match else 0

    # Reviewed (Swiss-Prot)
    is_reviewed = 1 if record.id.startswith("sp|") else 0

    # Append
    ids.append(clean_id)
    sequences.append(str(record.seq))
    pe_list.append(pe_val)
    taxon_list.append(ox_val)
    reviewed_list.append(is_reviewed)

df_seq = pd.DataFrame({
    "EntryID": ids,
    "sequence": sequences,
    "PE": pe_list,
    "TaxonomyID": taxon_list,
    "Reviewed": reviewed_list
})

print(f"   -> Loaded {len(df_seq)} sequences.")
print(f"   -> First PE: {pe_list[0]}, TaxID: {taxon_list[0]}")


# ================================
# 4️⃣ ĐỌC TERMS + GROUP LABELS
# ================================
print("\n2. Đang đọc Terms...")

df_terms = pd.read_csv(TRAIN_TERMS_PATH, sep="\t", usecols=["EntryID", "term"])

print("3. Đang gom nhóm labels...")
df_labels = df_terms.groupby("EntryID")["term"].apply(list).reset_index()


# ================================
# 5️⃣ GHÉP DỮ LIỆU
# ================================
print("4. Đang merge dữ liệu...")

df_final = pd.merge(df_seq, df_labels, on="EntryID", how="inner")

print("\n=== SAMPLE DATA ===")
display(df_final.head())

if len(df_final) > 0:
    sample = df_final.iloc[100]
    print("\n[SAMPLE DETAILS]")
    print(f"- EntryID:     {sample['EntryID']}")
    print(f"- TaxonomyID:  {sample['TaxonomyID']}")
    print(f"- PE:          {sample['PE']}")
    print(f"- Reviewed:    {sample['Reviewed']}")
    print(f"- Seq length:  {len(sample['sequence'])}")
    print(f"- GO Terms:    {sample['term'][:5]} (Total: {len(sample['term'])})")
else:
    print("❌ Không có dữ liệu sau khi merge — kiểm tra file EntryID.")


📁 Project Root: d:\vhproj\CAFA-6-Protein-Function-Prediction-Kaggle
📁 Train Folder: d:\vhproj\CAFA-6-Protein-Function-Prediction-Kaggle\data\Train
📄 FASTA Exists: True
📄 TERMS Exists: True

1. Đang đọc FASTA & trích xuất Features...
   -> Loaded 82404 sequences.
   -> First PE: 1, TaxID: 9606

2. Đang đọc Terms...
3. Đang gom nhóm labels...
4. Đang merge dữ liệu...

=== SAMPLE DATA ===


,EntryID,sequence,PE,TaxonomyID,Reviewed,term
0,A0A0C5B5G6,MRWQEMGYIFYPRKLR,1,9606,1,"[GO:0001649, GO:0033687, GO:0005615, GO:000563..."
1,A0JNW5,MAGIIKKQILKHLSRFTKNLSPDKINLSTLKGEGELKNLELDEEVL...,1,9606,1,"[GO:0120013, GO:0034498, GO:0005769, GO:012000..."
2,A0JP26,MVAEVCSMPAASAVKKPFDLRSKMGKWCHHRFPCCRGSGKSNMGTS...,1,9606,1,[GO:0005515]
3,A0PK11,MPGWFKKAWYGLASLLSFSSFILIIVALVVPHWLSGKILCQTGVDL...,1,9606,1,"[GO:0007605, GO:0005515]"
4,A1A4S6,MGLQPLEFSDCYLDSPWFRERIRAHEAELERTNKFIKELIKDGKNL...,1,9606,1,"[GO:0005829, GO:0010008, GO:0005515, GO:000509..."



[SAMPLE DETAILS]
- EntryID:     O14910
- TaxonomyID:  9606
- PE:          1
- Reviewed:    1
- Seq length:  233
- GO Terms:    ['GO:0005515', 'GO:0065003', 'GO:0006887', 'GO:0005886', 'GO:0097016'] (Total: 6)


In [ ]:
from pathlib import Path

# ===== 1️⃣ Xác định đúng project root như phần trên =====
def find_project_root(target_folder_name="CAFA-6-Protein-Function-Prediction-Kaggle"):
    path = Path.cwd()
    while path.name != target_folder_name:
        if path.parent == path:
            raise RuntimeError("❌ Không tìm thấy project root.")
        path = path.parent
    return path

PROJ_DIR = find_project_root("CAFA-6-Protein-Function-Prediction-Kaggle")

# ===== 2️⃣ Tạo đường dẫn file SAVE =====
SAVE_PATH = PROJ_DIR / "data" / "ver1.pkl"

print(f"📁 Đang lưu dữ liệu vào: {SAVE_PATH}")

# ===== 3️⃣ Lưu file bằng pickle =====
df_final.to_pickle(SAVE_PATH)

print("✅ Đã lưu thành công!")
print(f"👉 Lần sau chỉ cần: df = pd.read_pickle(r'{SAVE_PATH}')")


📁 Đang lưu dữ liệu vào: d:\vhproj\CAFA-6-Protein-Function-Prediction-Kaggle\data\processed_data.pkl
✅ Đã lưu thành công!
👉 Lần sau chỉ cần: df = pd.read_pickle(r'd:\vhproj\CAFA-6-Protein-Function-Prediction-Kaggle\data\processed_data.pkl')
